In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import cv2
import torch.utils.data as data
from glob import glob
from random import shuffle
from PIL import Image
import random
from tqdm import tqdm
from pred import predprob
from skimage import io 

In [2]:
import sys
paths = sys.path
sys.path.append('/home/yuyue/yuyue/Synchronized-BatchNorm-PyTorch-master')

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((512, 512)),
#         RandomCrop((512, 512)),
        transforms.RandomHorizontalFlip(),   # horizontal flip
        transforms.RandomVerticalFlip(),   # vertival flip
        transforms.ColorJitter(0.2,0.2,0.2,0.04),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # mean, std
    ]),
    'val': transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
}

In [5]:
df_test = pd.read_csv("/data/Pathology/SPIE/test_set/breastpathq-test/test_patient_ids.csv")


In [6]:
df_test.head()

,patient_id,slide
0,38,102174
1,39,102175
2,39,102176
3,39,102177
4,40,102178


In [7]:
from torch.nn import DataParallel
from densenet import densenet169
model_ft = densenet169(pretrained=False)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 1)
#model_ft.add_module("sigmoid", module=nn.Sigmoid())
from sync_batchnorm import convert_model
model_ft = convert_model(model_ft)
# load pretrained model
model_ft.load_state_dict(torch.load("/data/yuyue/SPIE/model_weight/densenet169_512_0626_sigmoid_1.pth"))
# print("pretrained model loaded")
model = model_ft.cuda()
model = model.train(False)

densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [8]:
val_list = glob("/data/Pathology/SPIE/training_set/breastpathq/datasets/validation/*.tif")
test_list = glob("/data/Pathology/SPIE/test_set/breastpathq-test/test_patches/*.tif")

In [9]:
df_val = pd.read_csv('/data/Pathology/SPIE/training_set/breastpathq/datasets/val_labels.csv') 

In [10]:
list_file = []
for path in test_list:
    file_name = path.split("/")[-1].strip(".tif")
    slide = int(file_name.split("_")[0])
    rid = int(file_name.split("_")[1])    
    inputs = io.imread(path)
    inputs = Image.fromarray(inputs)
    inputs = data_transforms['test'](inputs)
    inputs = np.array(inputs) 
    inputs = torch.from_numpy(inputs[np.newaxis, ...]).float()
    inputs = Variable(inputs).cuda()
    output = model(inputs)
    output = nn.Sigmoid()(output)
    output = output.data.cpu()
    output = float(output)
    #score = df_val[(df_val['slide']==slide) & (df_val['rid']==rid)]['y'].tolist()[0]
    list_file.append([slide,rid,output])

In [11]:
list_file

[[102178, 16, 0.007332372013479471],
 [102177, 11, 0.15741461515426636],
 [102180, 20, 0.0846862867474556],
 [102182, 8, 0.6979148983955383],
 [102174, 20, 0.7987700700759888],
 [102178, 46, 0.9091579914093018],
 [102840, 37, 0.000375798117602244],
 [102176, 6, 0.3615788519382477],
 [102185, 42, 0.07496723532676697],
 [102190, 5, 0.7533026337623596],
 [102839, 64, 0.3337755799293518],
 [102175, 45, 0.5998643636703491],
 [102197, 10, 0.17041783034801483],
 [102185, 39, 0.8591670393943787],
 [102190, 7, 0.7103330492973328],
 [102183, 13, 0.1623648703098297],
 [102176, 34, 0.6904687285423279],
 [102178, 45, 0.6806256175041199],
 [102176, 4, 0.17097537219524384],
 [102185, 26, 0.8812612891197205],
 [102181, 16, 0.5251662731170654],
 [102182, 36, 0.8164545893669128],
 [102177, 26, 0.34536299109458923],
 [102174, 34, 0.6330291032791138],
 [102187, 2, 0.5948101282119751],
 [102183, 9, 0.6544801592826843],
 [102185, 6, 0.5925828814506531],
 [102182, 17, 0.0712752714753151],
 [102562, 43, 0.653

In [13]:
df_result = pd.DataFrame(list_file, columns=['slide','rid','output']) 

In [14]:
df_result.head()

,slide,rid,output
0,102178,16,0.007332
1,102177,11,0.157415
2,102180,20,0.084686
3,102182,8,0.697915
4,102174,20,0.798770


In [36]:
output = df_result["output"].values.tolist()

In [37]:
score = df_result["score"].values.tolist()

In [38]:
from pred import predprob

In [41]:
 p_k = predprob(np.array(score),np.array(output))

In [42]:
p_k

0.9305546771235216